In [ ]:
from openai import OpenAI
from datetime import datetime, timezone, timedelta
import os
import base64
import json

# --- Cấu hình ---
AI_API_BASE = "https://api.thucchien.ai/v1"
AI_API_KEY = "sk-YzYUqhbM9WskmD2AQDUXJQ"
models = ["imagen-4"]

client = OpenAI(
    api_key=AI_API_KEY,
    base_url=AI_API_BASE
)

# --- Cấu hình log & thư mục ---
# LOG_FILE = "logs/full_log.jsonl"         # log chi tiết (dạng JSON Lines)
LOG_FILE_SUMMARY = "logs/summary.log"    # log hiển thị đẹp
IMAGE_DIR = "results_images"             # thư mục lưu ảnh
os.makedirs(IMAGE_DIR, exist_ok=True)
os.makedirs("logs", exist_ok=True)

# --- Cấu hình prompt ---
prompt = "a puppy"

# --- Thiết lập múi giờ Hà Nội (UTC+7) ---
hanoi_tz = timezone(timedelta(hours=7))
timestamp = datetime.now(hanoi_tz).strftime("%Y-%m-%d %H:%M:%S")
timestamp_safe = datetime.now(hanoi_tz).strftime("%Y%m%d_%H%M%S")

# --- Gọi API ---
response = client.images.generate(
    model=models[0],
    prompt=prompt,
    n=1,  # số ảnh cần tạo
    
)

# --- Lưu ảnh và ghi log ---
saved_images = []

for i, image_obj in enumerate(response.data):
    # Giải mã base64
    b64_data = image_obj.b64_json
    image_data = base64.b64decode(b64_data)
    
    # Tạo tên file theo timestamp
    image_filename = f"image_{timestamp_safe}_{i+1}.png"
    save_path = os.path.join(IMAGE_DIR, image_filename)
    
    # Lưu ảnh
    with open(save_path, 'wb') as f:
        f.write(image_data)
    
    print(f"✅ Ảnh {i+1} đã lưu tại: {save_path}")
    saved_images.append(save_path)

# --- Tạo log entry ---
log_entry = {
    "timestamp": timestamp,
    "prompt": prompt,
    "model": models[0],
    "saved_images": saved_images,
    "full_response": response.model_dump()  # toàn bộ dữ liệu trả về
}

# # --- Ghi log chi tiết (JSON Lines) ---
# with open(LOG_FILE, "a", encoding="utf-8") as f:
#     f.write(json.dumps(log_entry, ensure_ascii=False, indent=2))
#     f.write("\n")

# --- Ghi log tóm tắt (summary đẹp kiểu ChatGPT) ---
with open(LOG_FILE_SUMMARY, "a", encoding="utf-8") as f:
    f.write(f"\n=================== 🕒 {timestamp} (UTC+7) ===================\n\n")
    f.write("🎨 Prompt:\n")
    f.write(prompt + "\n\n")
    f.write(f"📊 Model: {models[0]}\n")
    f.write(f"🖼️ Ảnh đã lưu ({len(saved_images)}):\n")
    for path in saved_images:
        f.write(f"   → {path}\n")
    f.write("\n======================================================================\n")

# print(f"📝 Đã ghi log chi tiết vào {LOG_FILE}")
print(f"📝 Đã ghi log tóm tắt vào {LOG_FILE_SUMMARY}")
